In [56]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import automata.regex.regex as re
from automata.fa.dfa import DFA
# disable warnings
import warnings
warnings.filterwarnings('ignore')
# random graph generator
import random

def generate_random_graph(num_of_nodes, num_of_edges):
    nodes = [str(i) for i in range(1, num_of_nodes + 1)]
    send_edges = []
    receive_edges = []

    for i in range(num_of_edges):
        source = random.choice(nodes)
        if i == 0:
            source = '1'
        target = random.choice(nodes)
        while source == target:
            target = random.choice(nodes)
        letter = chr(97 + i)
        send_edges.append((source, target, letter))
        # nodes without source
        nodes_without_source = [node for node in nodes if node != source and node != '1']
        # random number of receivers
        num_of_receivers = random.randint(1, len(nodes_without_source) - 1)
        # choose num_of_receivers random nodes
        if num_of_receivers > 0:
            nodes_soucres = random.sample(nodes_without_source, num_of_receivers)
            for node_source in nodes_soucres:
                target_r = random.choice(nodes)
                if node_source != target_r:
                    receive_edges.append((node_source, target_r, letter))
    # remove duplicates
    send_edges = list(set(send_edges))
    receive_edges = list(set(receive_edges))
    return nodes, send_edges, receive_edges

# take a dfa edges and return the fitting regeular expression
def to_dfa(init, nodes, send_edges, receive_edges, final_states):
    # build transitions out of edges and letters set
    transitions = {}
    letters = set()
    for edge in send_edges:
        letters.add(edge[2])
    # init all transitions to be self referencing
    for node in nodes:
        transitions[node] = {}
        for letter in letters:
            transitions[node][letter] = node
    for edge in receive_edges:
        transitions[edge[0]][edge[2]] = edge[1]
    
    # build DFA
    dfa = DFA(
        states= nodes,
        input_symbols= letters,
        transitions= transitions,
        initial_state=init,
        final_states=final_states
    )   
    # build regex
    return dfa

# create a dfa for each (sending edge) * (nodes) combination
def create_dfa_list(nodes, send_edges, receive_edges):
    dfa_list = {}
    for end_node in nodes:
        for edge in send_edges:
            final_states = {end_node}
            dfa_list[(edge[1], end_node)] = to_dfa(edge[1], set(nodes), send_edges, receive_edges, final_states)
            dfa_list[(edge[0], end_node)] = to_dfa(edge[0], set(nodes), send_edges, receive_edges, final_states)
    return dfa_list

def eval_word(dfa_list, word, init='1'):
    node_sums = {}
    for node in nodes:
        sum = 0
        # print(node)
        for letter_index in range(len(word)):
            letter = word[letter_index]
            # print(letter)
            remaining_word = word[letter_index+1:]
            letter_send_edges = [edge for edge in send_edges if edge[2] == letter]
            for edge in letter_send_edges:
                a1 = dfa_list[(edge[1], node)].accepts_input(remaining_word)
                a2 = dfa_list[(edge[0], node)].accepts_input(remaining_word)
                # if a1 and (node == edge[1]):
                #     a1 = 0
                # if a2 and (node == edge[0]):
                #     a2 = 0
                # print((edge[1], node),(edge[0], node))
                # print(a1, a2)
                sum = sum + a1 - a2
        node_sums[node] = sum
    node_sums[init] = float('inf')
    return node_sums

In [57]:
def send_edge(letter, node_values, send_edges, receive_edges):
    # first, deal with sending edge
    for edge in send_edges:
        # if letter in edge, add to out
        if letter == edge[2]:
            if node_values[edge[0]][0] == 0:
                return 0
            node_values[edge[0]][0] -= 1 # deal with initial not out since sending overrules receiving
            node_values[edge[1]][1] += 1
            break
    # second, deal with receiving edges
    for edge in receive_edges:
        # if letter in edge, add to in
        if letter == edge[2]:
            node_values[edge[0]][2] = node_values[edge[0]][0]
            node_values[edge[1]][1] += node_values[edge[0]][0]
    # commit all in and out
    for node in nodes:
        node_values[node][0] = node_values[node][0] + node_values[node][1] - node_values[node][2]
        node_values[node][1] = 0
        node_values[node][2] = 0
    return 1

# simulate word over the BP
def simulate(word, nodes, send_edges, receive_edges):
    # init node_values
    node_values = {}
    for node in nodes:
        node_values[node] = [0, 0, 0] # [initial, in, out]
    node_values[nodes[0]] = [float('inf'), 0, 0]
    # for each letter in word
    for letter in word:
        # send letter
        if(send_edge(letter, node_values, send_edges, receive_edges) == 0):
            raise Exception('Letter ' + letter + ' has no cores')
    # return node_values first value in each node
    # convert to [(node, node_values[node][0]) for node in nodes] to dict
    return dict([(node, node_values[node][0]) for node in nodes])

def compare_outputs(word, nodes, send_edges, receive_edges):
    # get verifier output
    verifier_output = eval_word(dfa_list, word)
    # get simulate output
    simulate_output = simulate(word, nodes, send_edges, receive_edges)
    # compare outputs
    for node in nodes:
        if verifier_output[node] != simulate_output[node]:
            print(f"Word: {word}")
            print('verifier_output', verifier_output)
            print('simulate_output', simulate_output)
            print('nodes', nodes)
            print('send_edges', send_edges)
            print('receive_edges', receive_edges)
            draw_network_graph(nodes, send_edges, receive_edges)
            raise Exception(f"Node {node} verifier output {verifier_output[node]} != simulate output {simulate_output[node]}")
    # print('Y')

In [58]:
def calculate_sum_of_letter(word, letter):
    return len([l for l in word if l == letter])

def gen_word_run(word, node_values, send_edges, receive_edges):
    # search for non empty nodes, and rand some number of sending actions from the node
    non_empty_nodes = [node for node in nodes if node_values[node][0] > 0]
    # 
    possible_send_edges = [edge for edge in send_edges if edge[0] in non_empty_nodes]
    all_possible_letters = [edge[2] for edge in possible_send_edges]
    num_of_instances = [calculate_sum_of_letter(word, letter) for letter in all_possible_letters]
    # for each possible letter, give a nuumber
    letter_prob_numerator = [max(sum(num_of_instances) - num_of_instances[i], 1) for i in range(len(all_possible_letters))]
    letter_prob = [letter_prob_numerator[i] / sum(letter_prob_numerator) for i in range(len(letter_prob_numerator))]
    letter = np.random.choice(all_possible_letters, p=letter_prob)
    # get node of letter
    node = [edge[0] for edge in possible_send_edges if edge[2] == letter][0]
    #
    # node = np.random.choice(non_empty_nodes)
    # select number of sending actions, if float('inf') rand up to 10
    max_num_sending_actions = 8 if node_values[node][0] == float('inf') else node_values[node][0]
    # number of sending actions, randomized with favor to lower numbers
    num_sending_actions = np.random.randint(0, max_num_sending_actions)
    # select letters to send
    node_send_edges = [edge[2] for edge in send_edges if edge[0] == node]
    if len(node_send_edges) == 0:
        return ''
    letter = np.random.choice(node_send_edges)
    # send letters
    for i in range(num_sending_actions):
        if(send_edge(letter, node_values, send_edges, receive_edges) == 0):
            break
        word += letter
    return word

def gen_word(send_edges, receive_edges, max_runs):
    # simulate word over the BP
    node_values = {}
    for node in nodes:
        node_values[node] = [0, 0, 0] # [initial, in, out]
    node_values[nodes[0]] = [float('inf'), 0, 0]
    # set random number of runs out of max_runs
    num_of_runs = max_runs#np.random.randint(1, max_runs)
    # run gen_word_run num_of_runs times
    word = ''
    for i in range(num_of_runs):
        word = gen_word_run(word, node_values, send_edges, receive_edges)
    return word


In [59]:

def draw_network_graph(nodes, send_edges, receive_edges, node_pos = None, scale_factor = 1):
    # Create an empty directed graph
    graph = nx.DiGraph()

    # Add nodes to the graph
    graph.add_nodes_from(nodes)

    # if 2 edges have the same source and target, merge them to one edge
    send_edges = [(edge[0], edge[1], f'{edge[2]}!!') for edge in send_edges]
    receive_edges = [(edge[0], edge[1], f'{edge[2]}??') for edge in receive_edges]
    all_edges = send_edges + receive_edges

    # merge all edges labels
    merged_edges = []
    for edge in all_edges:
        found = False
        for e in merged_edges:
            if e[0] == edge[0] and e[1] == edge[1]:
                e[2] = e[2] + ', ' + edge[2]
                found = True
        if found == False:
            e = {
                0: edge[0],
                1: edge[1],
                2: edge[2],
            }
            merged_edges.append(e)

    # Set the layout of the graph, where every 2 nodes are in a row, from top to bottom
    scale = 6
    pos = {}
    if node_pos:
        for n in node_pos:
            pos[n] = (node_pos[n][0] / scale, node_pos[n][1] / scale)
    else:
        for i in range(len(nodes)):
            pos[nodes[i]] = ((-i % 2) / scale, -(i // 2) / scale)

    # Draw nodes
    nx.draw_networkx_nodes(graph, pos, node_color='lightblue', node_size=500)

    # Add edges to the graph
    for edge in merged_edges:
            graph.add_edge(edge[0], edge[1], label=edge[2], labeldistance=0.5)

    edges = graph.edges()
    nx.draw_networkx_edges(graph, pos, edgelist=edges, connectionstyle='arc3,rad=0.05', arrowsize=20)

    # Draw edge labels, above the edges and not overlapping with them
    # create a dictionary of tuble pos to label
    for edge in edges:
        e = graph.get_edge_data(edge[0], edge[1])
        x = (pos[edge[0]][0] + pos[edge[1]][0]) / 2
        y = (pos[edge[0]][1] + pos[edge[1]][1]) / 2
        # if left to right edge, subtract 0.05 to y
        if pos[edge[0]][0] < pos[edge[1]][0]:
            y -= 0.02 * scale_factor
        # if right to left edge, add 0.05 from y
        elif pos[edge[0]][0] > pos[edge[1]][0]:
            y += 0.02 * scale_factor
        # adjust x axis
        if pos[edge[0]][1] < pos[edge[1]][1]:
            x += 0.01
        elif pos[edge[0]][1] > pos[edge[1]][1]:
            x -= 0.01
        # if self loop
        if pos[edge[0]][0] == pos[edge[1]][0] and pos[edge[0]][1] == pos[edge[1]][1]:
            y += 0.03
        plt.text(x, y, e.get('label'), horizontalalignment='center', fontsize=12)

    # Draw node labels
    node_labels = {node: node for node in nodes}
    nx.draw_networkx_labels(graph, pos, labels=node_labels)

    # Display the graph
    plt.axis('off')
    # save the graph
    plt.show()

In [60]:

for j in range(20):
    # show random graph
    # random number of nodes and edges in exponential scale
    number_of_nodes = random.randint(5, 20)
    number_of_edges = random.randint(number_of_nodes - 2, number_of_nodes + 2)

    nodes, send_edges, receive_edges = generate_random_graph(number_of_nodes, number_of_edges)
    dfa_list = create_dfa_list(nodes, send_edges, receive_edges)

    # gen n words and compare outputs
    for i in range(100):
        word = gen_word(send_edges, receive_edges, number_of_edges * 2)
        # print(f"Word {i}: {word}")
        compare_outputs(word, nodes, send_edges, receive_edges)
    print('Success for graph', j, 'with', len(nodes), 'nodes and', len(send_edges), 'edges')

Success for graph 0 with 16 nodes and 18 edges
Success for graph 1 with 17 nodes and 19 edges
Success for graph 2 with 15 nodes and 13 edges
Success for graph 3 with 18 nodes and 18 edges
Success for graph 4 with 14 nodes and 15 edges
Success for graph 5 with 15 nodes and 13 edges
Success for graph 6 with 13 nodes and 14 edges
Success for graph 7 with 7 nodes and 7 edges
Success for graph 8 with 15 nodes and 17 edges
Success for graph 9 with 13 nodes and 13 edges
Success for graph 10 with 12 nodes and 14 edges
Success for graph 11 with 17 nodes and 17 edges
Success for graph 12 with 11 nodes and 9 edges
Success for graph 13 with 15 nodes and 13 edges
Success for graph 14 with 12 nodes and 11 edges
Success for graph 15 with 11 nodes and 12 edges
Success for graph 16 with 14 nodes and 14 edges
Success for graph 17 with 8 nodes and 8 edges
Success for graph 18 with 5 nodes and 3 edges
Success for graph 19 with 17 nodes and 19 edges
